# Zonas Medellin

In [10]:
import geopandas as gpd
import osmnx as ox
import utils.analysis as an

# List of areas of interest
zo = ['floresta_HH','moravia_LH','aguacatala_HL']
z = zo[0]

# Show the zone that is running
print(f"Zona ejecutada: {z}")

# Load nodes and edges files
nodes = gpd.read_file(f'../output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
edges = gpd.read_file(f'../output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')

# Add 'key' column to edges
edges['key'] = 0

# Set indexes
nodes = nodes.set_index(['osmid'])
edges = edges.set_index(['u', 'v', 'key'])

# DEM file path
DEM_path = (f'../Input/raster/medellin_DEM.tif')

# Calculate elevation
nodes, edges = an.elevation_DEM(nodes, edges, DEM_path)

# Show the first 10 rows of nodes and edges
print("Primeras 10 filas de nodes:")
print(nodes.head(10))

print("\nPrimeras 10 filas de edges:")
print(edges.head(10))

# Filter and show edges with length 0
edges_con_longitud_cero = edges[edges['length'] == 0]
print("\nEdges con longitud igual a 0:")
print(edges_con_longitud_cero)

nodes.to_file(f'../output/shape/slope/{z}/{z}_slope_nodes.shp')
edges.to_file(f'../output/shape/slope/{z}/{z}_slope_edges.shp')


Primeras 10 filas de nodes:
               x         y    elevation                   geometry
osmid                                                             
0     -75.565477  6.269190  1481.780029  POINT (-75.56548 6.26919)
2     -75.565782  6.267539  1483.645386  POINT (-75.56578 6.26754)
49    -75.564614  6.268904  1486.783936   POINT (-75.56461 6.2689)
40    -75.565006  6.272216  1470.312866  POINT (-75.56501 6.27222)
1     -75.566188  6.265307  1480.258545  POINT (-75.56619 6.26531)
186   -75.566355  6.264403  1481.211792   POINT (-75.56635 6.2644)
222   -75.566015  6.266285  1483.909912  POINT (-75.56602 6.26628)
223   -75.564871  6.267493  1489.059692  POINT (-75.56487 6.26749)
423   -75.565855  6.267142  1484.579224  POINT (-75.56586 6.26714)
3     -75.573253  6.273303  1450.000000   POINT (-75.57325 6.2733)

Primeras 10 filas de edges:
             osmid                                           geometry  \
u   v   key                                                       